FLooPy
======
> *Flow-based Loops with Python*

*FLooPy* is a **tester-agnostic sequencer** for **hardware-testing.** It has a **low-code** approach in which the same test can be performed within different **test-environments.**

#### Tests
* ... can organized in **hierarchical** layers with (sub-) tasks
* Every variable
    - ... has an unique name via **namespace**
    - ... is stored as a **time-signal** with timestamps
    - ... can be **sub-classed**
* Data dependencies between functions are resolved by a graph-based approach
* Auto-Save for loop-recovery and post-processing (experimental)
* Can handle **non-reentrant** tests by an extra extension (unpublished)

#### Loops
* Flexible **loop configuration** (nested, zipped, concatenated)
* Standard loops: `loop_items`, `loop_lin`, `loop_log`, `loop_bisect`
* Feedback for HiL possible

---

## Test-Structure by Example

For a Getting-Started example look into the *[FLooPy-Tutorial](./tutorial.ipynb)*.

Suppose we want to test if the internal resistance of an battery `rbat` is inside the test-limits:

* The test-environment is abstracted by the namespace of the `dut` input and must be specified later, just before running the test-case.
* The loop over the load-current can be either configured locally inside the test-case or later, just before running the test-case.

#### Test-Case

In [1]:
import floopy as fly


class Test_Rbat_Charged(fly.Task):
    dut = fly.Input()  # object to stimulate device-under-test and measure response
   
    i_load = fly.Input(min=0, max=1.5, unit='A', default=fly.loop_lin(num=5))
    
    def task(dut, i_load):
        dut.current = i_load
        return dut.voltage

    def final_fit(i=fly.Squeeze(task.i_load), v=fly.Squeeze(task)):
        import numpy as np
        return np.polyfit(i, v, deg=1)
        
    @fly.Output(min=0, ltl=0.2, utl=2, max=10, unit='ohm')
    def rbat(coeffs=final_fit):
        return coeffs[0]

#### Test-Plan

Now, the test-case `Test_Rbat_Charged` can be used with different loop-configurations and within different test-environments. For demonstration we just use the simple resistor equation as a simulation-environment.

In [2]:
class TestPlan_BatterCheck(fly.Task):
    def dut():
        class Dut:
            @property
            def voltage(self):
                resistance = 0.8  # ohm
                return resistance * self.current
        return Dut()

    test_rbat_charged = Test_Rbat_Charged(dut, i_load=fly.loop_log(0.1, 1, num=3))

In [3]:
dm = fly.DataManager()
dm.run_log(TestPlan_BatterCheck)

[22:35:02]  test_rbat_charged.task | test_rbat_charged.i_load=0.1

[22:35:02]      = 0.08000000000000002

[22:35:02]  test_rbat_charged.task | test_rbat_charged.i_load=0.31622776601683794

[22:35:02]      = 0.2529822128134704

[22:35:02]  test_rbat_charged.task | test_rbat_charged.i_load=1.0

...

[22:35:02]      = 0.7999999999999999

[22:35:02]  test_rbat_charged.rbat.check

[22:35:02]      = True

Summery: 1 passed

File: dm/dm_2024-10-08_22:35:02.json

For a real measurement we just need to replace the `dut` function with something like

```python
def dut():
    import instruments
    dut = instruments.PowerSupply()
    return dut
```

assuming an equal namespace

```python
dut.voltage
dut.current
```

with an equal behavior.

*Further information can be found in the **[FLooPy-Tutorial](./tutorial.ipynb)** which is more detailed.*

---

## Install


Simply do either

    pip install floopy

or download the repository for a more recent version, change into
the `floopy` folder and

    pip install . --user

If necessary you can run the tests with

    python -m pytest

## Licence

LoopyPlot is licenced under GPL 3.